In [1]:
import snowflake.connector
import pandas as pd
from dotenv import load_dotenv
import os

# Charger les variables d'environnement
load_dotenv()

# Configuration Snowflake
config = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

In [2]:
conn = snowflake.connector.connect(**config)
cursor = conn.cursor()
cursor.execute("SELECT CURRENT_DATABASE(), CURRENT_SCHEMA()")
current_db, current_schema = cursor.fetchone()
print(f"\n📂 Base de données actuelle: {current_db}")
print(f"📁 Schéma actuel: {current_schema}")


📂 Base de données actuelle: RAW_DATA_INTERN
📁 Schéma actuel: SILVER


In [3]:
cursor.execute("""
SELECT 
    TABLE_NAME,
    TABLE_TYPE,
    ROW_COUNT,
    BYTES,
    CREATED,
    LAST_ALTERED
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = CURRENT_SCHEMA()
ORDER BY TABLE_NAME
""")
cursor.fetchall()

[('DEMO_H3_L10_CSP',
  'BASE TABLE',
  574550,
  4836864,
  datetime.datetime(2025, 6, 14, 7, 10, 15, 597000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  datetime.datetime(2025, 6, 14, 7, 10, 19, 58000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>)),
 ('DEMO_H3_LVL10_SUP10KHAB',
  'BASE TABLE',
  574667,
  3180032,
  datetime.datetime(2025, 6, 14, 6, 25, 8, 786000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  datetime.datetime(2025, 6, 14, 7, 7, 10, 832000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>)),
 ('DEMO_SEG',
  'BASE TABLE',
  99999,
  1179648,
  datetime.datetime(2025, 6, 14, 13, 25, 55, 259000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  datetime.datetime(2025, 6, 14, 13, 25, 56, 344000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>)),
 ('DEMO_SEG_AB',
  'BASE TABLE',
  40000,
  444928,
  datetime.datetime(2025, 6, 14, 13, 25, 51, 469000, tzinf

In [7]:
def get_cursor():
    conn = snowflake.connector.connect(**config)
    cursor = conn.cursor()
    return cursor


def get_table(table_name="DEMO_SEG_CLIENT", limit=10):
    if limit > 0:
        cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
    else:
        cursor.execute(f"SELECT * FROM {table_name}")
    data = cursor.fetch_pandas_all()
    return data   

type(get_table())

pandas.core.frame.DataFrame